## 5.7 이웃기반 협업 필터링
이웃기반 협업필터링의 아이디어는 비슷한 사용자들은 비슷한 취향과 선호를 가진다는 점에서 그들의 과거 레이팅은 특정 사용자의 미래 레이팅을 예측하는데 사용 할 수 있다는 점

#### 사용자 기반 협업 필터링
*유사한 이웃*의 해당 아이템에 대한 rating을 가중평균

* *이웃*: 해당 아이템을 평가한 사용자들
* *유사한* : 두 사용자(u, v) 가 동시에 평가한 아이템갯수의 dim인 rating vector의 유사도(e.g. pearson coef)

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('movie_baseline.csv', index_col=0)
data.values.astype('I')
ratings = data.copy(deep=True)
ratings_view = ratings.style.highlight_null("orange").set_precision(2)
ratings_view

,포레스트 검프,타이타닉,대부,배트맨,매트릭스,에일리언
u1,5.00,4.00,nan,1.00,2,1.00
u2,4.00,nan,3.00,1.00,1,2.00
u3,nan,5.00,5.00,nan,3,3.00
u4,2.00,nan,1.00,4.00,5,4.00
u5,2.00,2.00,2.00,nan,4,nan
u6,1.00,2.00,1.00,nan,5,4.00


먼저, 사용자들의 유사도 행렬 `sim_mat`을 만들어준다. 
* `co_rated`:  두 사용자 둘다 평가한 아이템 평점 집합 (I<sub>uv</sub>)

![](cf_user_sim.PNG)

In [2]:
def std_cancled(a, a_mu):
    return np.sqrt(np.sum(np.square(a - a_mu)))

def sim(u, v):
    co_rated = ~np.isnan(u) & ~np.isnan(v)
    u = u[co_rated]
    v = v[co_rated]
    u_mu = u.mean()
    v_mu = v.mean()
    nom = np.sum((u - u_mu) * (v - v_mu))
    denom = std_cancled(u, u_mu) * std_cancled(v, v_mu)
    return nom / denom

In [3]:
nu, mi = data.shape

sim_mat = np.zeros([nu, nu])

for n in range(nu):
    for k in range(n+1, nu):
        u = data.loc[f'u{n+1}'].values
        v = data.loc[f'u{k+1}'].values
        sim_mat[n, k] = sim(u, v).round(2)
            
sim_mat = sim_mat + sim_mat.T # symmetric
np.fill_diagonal(sim_mat, 1) # self-coef always 1
print(sim_mat)

[[ 1.    0.87  0.94 -0.8  -0.94 -0.9 ]
 [ 0.87  1.    0.87 -0.84 -0.94 -0.94]
 [ 0.94  0.87  1.   -0.97 -1.   -0.95]
 [-0.8  -0.84 -0.97  1.    0.97  0.97]
 [-0.94 -0.94 -1.    0.97  1.    0.97]
 [-0.9  -0.94 -0.95  0.97  0.97  1.  ]]


계산한 유사도가 일부 책과 다르게 나온다.
원 저자의 [github](https://github.com/ikatsov/tensor-house/blob/master/recommendations/user-based-cf.ipynb)을 보면, 피어슨 유사도 행렬이 asymmetric하게 나오는데, 뭔가 원 저자가 계산을 잘못한건 아닌지... 싶다. 피어슨 유사도에서 교환법칙이 성립 안할수가 있는것인가..?

목표 사용자 `u`의 아이템 `i`에 대한 예상 평점은, 
 * 아이템 `i`를 평가한 사용자중
 * 가장 유사도가 큰 `k` 이웃을 골라
 * 그 이웃의 해당 아이템에 대한 평점을 유사도 가중평균을 해준 값에
 * 사용자의 baseline 평점을 합해준다.
 
 ![](cf_user_rhat.PNG)

In [68]:
def predict(u, i, k):
    """
    u: user index 
    i: item index
    k: number of closest neighborhoods
    """
    r = data.values    
    weighted_sum = 0
    normalizer = 0
    for v in range(nu):
        if v == u:
            continue
        if not np.isnan(r[v, i]): # if v rated i
            w = sim_mat[u, v]
            weighted_sum += w * (r[v, i] - np.nanmean(r[v]))
            normalizer += np.abs(w)
    return (np.nanmean(r[u]) + weighted_sum / normalizer).round(2)

예를 들어 1번 사용자와 대부 영화에 대한 빠진 레이팅을 이웃 크기 2를 사용해 예측해보자.

In [69]:
predict(0, 2, 2)

3.79

전체 완성.

In [70]:
for u in range(nu):
    for i in range(mi):
        if np.isnan(ratings.iloc[u, i]):
            ratings.iloc[u, i] = predict(u, i, 2)

In [71]:
ratings_view

,포레스트 검프,타이타닉,대부,배트맨,매트릭스,에일리언
u1,5.00,4.00,3.79,1.00,2,1.00
u2,4.00,3.06,3.00,1.00,1,2.00
u3,5.48,5.00,5.00,2.80,3,3.00
u4,2.00,2.35,1.00,4.00,5,4.00
u5,2.00,2.00,2.00,3.70,4,3.50
u6,1.00,2.00,1.00,3.79,5,4.00


역시나, 원저자의 계산과는 다르다. (틀린 코드 있으면 제보 바랍니다.)